## Tiền xử lý dữ liệu
- Dữ liệu được phân tích trong dự án Data Mining này là dữ liệu ghi nhận các phim Anine (hoạt hình Nhật Bản) và các thông tin về đánh giá của các người dùng về nó. Dựa vào dữ liệu này ta có thể rút ra một số đánh giá về hành vi hay thói quen của người dùng khi lựa chọn xem phim Anime theo sở thích của họ (Ví dụ: người thích xem Conan thì cũng thích xem Doreamon).

- Phân tích bảng __anime.csv__, tìm những bất thường như duplicate, null, loại bỏ những dữ liệu thừa

In [1]:
import pandas as pd

anime_dataset = pd.read_csv("Dataset/anime.csv")
anime_dataset.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


- Thông tin về các cột dữ liệu trong bảng

In [2]:
anime_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


- có sự ít hơn về mặt dữ liệu non-null ở __gerne__, __type__, __rating__.

- Kiểm tra duplicated records

In [3]:
anime_dataset.duplicated().value_counts()

False    12294
Name: count, dtype: int64

- Hoàn toàn không có dữ liệu trùng lặp
- Kiểm tra số lượng giá trị null ở các cột

In [4]:
anime_dataset.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

- Dữ liệu null rất ít, không cần phải dùng thuật toán cho việc fill dữ liệu mà xóa hẳn các hàng có dữ liệu null luôn

In [5]:
anime_dataset = anime_dataset.dropna()

In [6]:
anime_dataset.shape

(12017, 7)

In [7]:
anime_dataset.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


- Dữ liệu đã được làm sạch. Tuy nhiên, cần xử lý sâu vào dữ liệu để các bước __Khai Phá__ phía sau thực hiện tốt hơn.
- Nhìn vào cột Genre, ta thấy dữ liệu đang là chuỗi các __Genre__ được ngăn cách bởi dấu phẩy. Nếu để như vậy, thì việc áp dụng các thuật toán, machine learning sẽ không khả thi, vì các thuật toán thực hiện dựa trên ngôn ngữ tự nhiên cho dữ liệu dạng này không hiệu quả, nên ta sẽ chuyển dữ liệu cột __Genre__ thành dạng list có thể truy cập và tính toán được trong các model.

In [8]:
import html, re
from sklearn.preprocessing import MultiLabelBinarizer

anime_dataset = anime_dataset.copy()

# chuẩn hoá chuỗi
anime_dataset['genre'] = anime_dataset['genre'].fillna('Unknown').astype(str).map(html.unescape)

# thay nhiều dấu phẩy / dấu chấm phẩy về phẩy, xoá khoảng trắng thừa
anime_dataset['genre'] = (anime_dataset['genre']
               .str.replace(r'[;/|]', ',', regex=True)
               .str.replace(r'\s*,\s*', ',', regex=True)
               .str.strip(', ')
)

# chuẩn hoá viết thường (tuỳ nhu cầu trực quan có thể giữ hoa đầu)
anime_dataset['genre_norm'] = anime_dataset['genre'].str.lower()

In [9]:
def to_list(s):
    if s in ('', 'unknown', 'nan'):
        return ['unknown']
    # tách, bỏ rỗng, bỏ trùng, giữ thứ tự xuất hiện
    items = [x.strip() for x in s.split(',') if x.strip()]
    dedup = list(dict.fromkeys(items))
    return dedup or ['unknown']

anime_dataset['genre_list'] = anime_dataset['genre_norm'].apply(to_list)


In [10]:
anime_dataset.head()

,anime_id,name,genre,type,episodes,rating,members,genre_norm,genre_list
0,32281,Kimi no Na wa.,"Drama,Romance,School,Supernatural",Movie,1,9.37,200630,"drama,romance,school,supernatural","[drama, romance, school, supernatural]"
1,5114,Fullmetal Alchemist: Brotherhood,"Action,Adventure,Drama,Fantasy,Magic,Military,...",TV,64,9.26,793665,"action,adventure,drama,fantasy,magic,military,...","[action, adventure, drama, fantasy, magic, mil..."
2,28977,Gintama°,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.25,114262,"action,comedy,historical,parody,samurai,sci-fi...","[action, comedy, historical, parody, samurai, ..."
3,9253,Steins;Gate,"Sci-Fi,Thriller",TV,24,9.17,673572,"sci-fi,thriller","[sci-fi, thriller]"
4,9969,Gintama&#039;,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.16,151266,"action,comedy,historical,parody,samurai,sci-fi...","[action, comedy, historical, parody, samurai, ..."


In [11]:
# số lượng & primary
anime_dataset['n_genres'] = anime_dataset['genre_list'].str.len()
anime_dataset['genre_primary'] = anime_dataset['genre_list'].str[0].astype('category')

# explode cho EDA
df_expl = anime_dataset.explode('genre_list', ignore_index=True)
df_expl['genre_list'] = df_expl['genre_list'].astype('category')

# one-hot multi-label
mlb = MultiLabelBinarizer()
genre_ohe = pd.DataFrame(
    mlb.fit_transform(anime_dataset['genre_list']),
    columns=[g.replace(' ', '_') for g in mlb.classes_],
    index=anime_dataset.index
)
df_ml = pd.concat([anime_dataset, genre_ohe], axis=1)


In [12]:
df_ml

,anime_id,name,genre,type,episodes,rating,members,genre_norm,genre_list,n_genres,...,shounen_ai,slice_of_life,space,sports,super_power,supernatural,thriller,vampire,yaoi,yuri
0,32281,Kimi no Na wa.,"Drama,Romance,School,Supernatural",Movie,1,9.37,200630,"drama,romance,school,supernatural","[drama, romance, school, supernatural]",4,...,0,0,0,0,0,1,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,"Action,Adventure,Drama,Fantasy,Magic,Military,...",TV,64,9.26,793665,"action,adventure,drama,fantasy,magic,military,...","[action, adventure, drama, fantasy, magic, mil...",7,...,0,0,0,0,0,0,0,0,0,0
2,28977,Gintama°,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.25,114262,"action,comedy,historical,parody,samurai,sci-fi...","[action, comedy, historical, parody, samurai, ...",7,...,0,0,0,0,0,0,0,0,0,0
3,9253,Steins;Gate,"Sci-Fi,Thriller",TV,24,9.17,673572,"sci-fi,thriller","[sci-fi, thriller]",2,...,0,0,0,0,0,0,1,0,0,0
4,9969,Gintama&#039;,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.16,151266,"action,comedy,historical,parody,samurai,sci-fi...","[action, comedy, historical, parody, samurai, ...",7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,hentai,[hentai],1,...,0,0,0,0,0,0,0,0,0,0
12290,5543,Under World,Hentai,OVA,1,4.28,183,hentai,[hentai],1,...,0,0,0,0,0,0,0,0,0,0
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,hentai,[hentai],1,...,0,0,0,0,0,0,0,0,0,0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,hentai,[hentai],1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
genre_onehot = pd.concat([anime_dataset['anime_id'], genre_ohe], axis=1)

- Ma trận thể hiện thể loại của phim

In [14]:
genre_onehot.head()

,anime_id,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,...,shounen_ai,slice_of_life,space,sports,super_power,supernatural,thriller,vampire,yaoi,yuri
0,32281,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5114,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,28977,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9253,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9969,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from functools import reduce

In [16]:
num_type = {}

for col in genre_ohe.columns:
    num_type[col] = reduce(lambda acc, cur: acc + cur, genre_ohe[col], 0)

In [17]:
num_categories = pd.DataFrame([num_type])
num_categories

,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,game,...,shounen_ai,slice_of_life,space,sports,super_power,supernatural,thriller,vampire,yaoi,yuri
0,2768,2316,72,4575,238,287,1977,628,2242,177,...,62,1204,377,533,451,1001,86,100,38,41


In [18]:
num_categories_T = num_categories.T
num_categories_T.columns = ["count"]  # đổi tên cột
num_categories_T.head()

,count
action,2768
adventure,2316
cars,72
comedy,4575
dementia,238


- 10 thể loại chiếm nhiều nhất trong dữ liệu

In [19]:
num_categories_T.sort_values("count", ascending=False).head(10)

,count
comedy,4575
action,2768
adventure,2316
fantasy,2242
sci-fi,2036
drama,1977
shounen,1683
kids,1598
romance,1437
slice_of_life,1204


- 10 Thể loại xuất hiện nhiều nhất trong dữ liệu

In [20]:
anime_dataset.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members',
       'genre_norm', 'genre_list', 'n_genres', 'genre_primary'],
      dtype='object')

In [21]:
anime_dataset = anime_dataset.drop(columns=['genre_norm', 'genre_list'])

- Lưu lại số liệu đã xử lý

In [22]:
genre_onehot.to_csv("Dataset/matrix_genre.csv", index=False)
anime_dataset.to_csv("Dataset/preprocessed_anime.csv", index=False)

- Đã chạy lại (1)